In [1]:
!pip install -qq langchain==0.3.14
!pip install -qq langchain-openai==0.3.0
!pip install -qq langchain-community==0.3.14

In [2]:
!pip install -qq markitdown

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
WEATHER_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=3,  # Reduced from 5 to 3
                                  search_depth='basic',  # Changed from 'advanced' to 'basic'
                                  include_answer=True,  # Include answers to reduce need for content extraction
                                  include_raw_content=False)  # Disable raw content to save tokens
# certain websites won't let you crawl them unless you specify a user-agent
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []
    max_content_length = 1500  # Limit content per URL to prevent context overflow

    def extract_content(url):
        """Helper function to extract content from a URL."""
        try:
            extracted_info = md.convert(url)
            text_title = extracted_info.title.strip() if extracted_info.title else "No title"
            text_content = extracted_info.text_content.strip() if extracted_info.text_content else "No content"
            
            # Truncate content to prevent context overflow
            if len(text_content) > max_content_length:
                text_content = text_content[:max_content_length] + "... [Content truncated]"
            
            return f"{text_title}\n{text_content}"
        except Exception as e:
            return f"Error extracting content: {str(e)}"
    
    # parallelize execution of different urls (limit to 3 to reduce total content)
    limited_results = results[:3] if len(results) > 3 else results
    with ThreadPoolExecutor() as executor:
        for result in tqdm(limited_results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 10 seconds for the task to complete (reduced from 15)
                content = future.result(timeout=10)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
                docs.append(f"Timeout extracting from: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")
                docs.append(f"Error extracting from: {result['url']}")

    return docs


@tool
def get_weather(query: str) -> str:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    
    # Check if API key is properly set
    if WEATHER_API_KEY == 'your_weather_api_key_here':
        return "Weather API key not configured. Please set WEATHER_API_KEY environment variable."
    
    try:
        base_url = "http://api.weatherapi.com/v1/current.json"
        complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

        response = requests.get(complete_url, timeout=10)
        data = response.json()
        
        if data.get("location") and data.get("current"):
            location = data["location"]
            current = data["current"]
            weather_info = f"""
            Weather for {location['name']}, {location['region']}, {location['country']}:
            - Temperature: {current['temp_c']}°C ({current['temp_f']}°F)
            - Condition: {current['condition']['text']}
            - Humidity: {current['humidity']}%
            - Wind: {current['wind_kph']} kph
            - Feels like: {current['feelslike_c']}°C
            """
            return weather_info.strip()
        else:
            return f"Weather data not found for location: {query}"
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather data: {str(e)}"
    except Exception as e:
        return f"Unexpected error getting weather: {str(e)}"

In [6]:
get_weather.invoke("Get weather in kolkata now")

Calling weather tool


'Weather Data Not Found'

In [7]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

chatgpt_with_tools = chatgpt.bind_tools(tools)

In [26]:
prompt = "Show me Local Attractions in Bangalore and The Weather in Bangalore"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'search_web_extract_info',
  'args': {'query': 'Local Attractions in Bangalore'},
  'id': 'call_7SwuPH7BZUYE5eHFJA8Upxng',
  'type': 'tool_call'},
 {'name': 'get_weather',
  'args': {'query': 'Bangalore'},
  'id': 'call_CHqk48QDiDmOsGCMLp3B4NUV',
  'type': 'tool_call'}]

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """You are a helpful travel assistant. For location queries, provide local attractions as bullet points and current weather.

Tools available:
- get_weather: Get current weather for a location
- search_web_extract_info: Search web for specific information

Keep responses concise and well-formatted."""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [28]:
from langchain.agents import create_tool_calling_agent

chatgpt = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

In [34]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='generate',
                               max_iterations=3,
                               handle_parsing_errors=True)

In [35]:
# Test with a simpler, more specific query to avoid context overflow
query = "Top 3 attractions in Paris and current weather"
try:
    resp = agent_executor.invoke({"query": query})
    print("\nResponse:")
    print(resp["output"])
except Exception as e:
    print(f"Error: {e}")
    print("Try using a weather API key or simplifying the query")

Calling web search tool


100%|██████████| 3/3 [00:05<00:00,  1.75s/it]


Calling weather tool
Calling weather tool

Response:
### Top 3 Attractions in Paris
1. **Eiffel Tower**
   - Iconic symbol of Paris with stunning views from the top.
   
2. **Louvre Museum**
   - Home to thousands of works of art, including the Mona Lisa.

3. **Notre-Dame Cathedral**
   - A masterpiece of French Gothic architecture, currently under restoration.

### Current Weather in Paris
Unfortunately, I couldn't retrieve the current weather data for Paris. You may want to check a reliable weather website or app for the latest updates.


In [31]:
# Test individual tools
print("Testing weather tool:")
weather_result = get_weather.invoke("Paris")
print(weather_result)

print("\n" + "="*50 + "\n")

print("Testing search tool:")
search_result = search_web_extract_info.invoke("top attractions in Paris")
for i, doc in enumerate(search_result[:2]):  # Show only first 2 results
    print(f"Result {i+1}:")
    print(doc[:200] + "..." if len(doc) > 200 else doc)
    print("-" * 30)


Testing weather tool:
Calling weather tool
Weather data not found for location: Paris


Testing search tool:
Calling web search tool


100%|██████████| 3/3 [00:03<00:00,  1.28s/it]

Result 1:
THE 15 BEST Things to Do in Paris (2025) - Must-See Attractions
![Advertisement](data:image/gif;base64...)

Skip to main content

[![Tripadvisor](https://static.tacdn.com/img2/brand_refresh_2025/logos...
------------------------------
Result 2:
The absolute must-do things in Paris - and what to avoid - Blogger at Large
[Skip to Content](#content)

Search

Magnifying Glass

Search for:

Close Search
×

* [Home](https://bloggeratlarge.com//)
*...
------------------------------


In [36]:
print(resp)

{'query': 'Top 3 attractions in Paris and current weather', 'output': "### Top 3 Attractions in Paris\n1. **Eiffel Tower**\n   - Iconic symbol of Paris with stunning views from the top.\n   \n2. **Louvre Museum**\n   - Home to thousands of works of art, including the Mona Lisa.\n\n3. **Notre-Dame Cathedral**\n   - A masterpiece of French Gothic architecture, currently under restoration.\n\n### Current Weather in Paris\nUnfortunately, I couldn't retrieve the current weather data for Paris. You may want to check a reliable weather website or app for the latest updates."}


In [37]:
from IPython.display import display, Markdown
display(Markdown(response['output']))

TypeError: 'AIMessage' object is not subscriptable